<a href="https://colab.research.google.com/github/lagarcian/ProyectodeGradoLuisGarcia/blob/main/DiferenciasFinitas2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import sympy as sp
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator

In [ ]:

#solución manufacturada u_m

def u_m(x, y):
    return x * np.cos((np.pi/2) * x) * y * np.cos((np.pi/2) * y)

def u_x(x, y, h=1e-5):
    return (u_m(x + h, y) - u_m(x - h, y)) / (2 * h)

def u_xx(x, y, h=1e-5):
    return (u_x(x + h, y, h) - u_x(x - h, y, h)) / (2 * h)

def u_y(x, y, h=1e-5):
    return (u_m(x , y + h) - u_m(x , y - h)) / (2 * h)

def u_yy(x, y, h=1e-5):
    return (u_y(x , y + h, h) - u_y(x , y - h, h)) / (2 * h)

def Laplaciano(x,y):
  return u_xx(x,y)+u_yy(x,y)

def f(x,y):
  return -Laplaciano(x,y)


resultado = f(0.5,0.5)

print(resultado)


x_val = 0.5
y_val = 0.5

result = u_xx(x_val, y_val)
print(result)

In [ ]:

#Aqui calculamos el laplaciano en símbolos para poder hallar la función fuente f(x,y), luego se evalúa numericamente para poder compararla
#con el método de diferencias finitas.
x, y = sp.symbols('x y', real=True)

u_m = x * sp.cos((sp.pi/2) * x) * y * sp.cos((sp.pi/2) * y)

u_x = sp.diff(u_m, x)
u_y = sp.diff(u_m, y)

#print(u_x)
#print(u_y)

u_xx= sp.diff(u_x, x)
u_yy=sp.diff(u_y,y)
Laplaciano = u_xx + u_yy
print(-Laplaciano)
#print(u_xx)
#print(u_yy)

x_val = 0.5
y_val = 0.5

u_x_val = u_x.subs([(x, x_val), (y, y_val)])
u_y_val = u_y.subs([(x, x_val), (y, y_val)])

u_xx_val = u_xx.subs([(x, x_val), (y, y_val)])
u_yy_val = u_yy.subs([(x, x_val), (y, y_val)])

Lap = u_xx_val+u_yy_val
res_val = -Lap
print(res_val) #Aqui se evalúa numéricamente
print(res_val.evalf()) #Aquí está su valor aproximado con decimales.









In [ ]:
#Aqui a partir de la solución manufacturada u_ma(x,y) hallamos la función fuente f(x,y)
def u_ma(x,y):
  return x * np.cos((np.pi/2) * x) * y * np.cos((np.pi/2) * y)

def f(x,y,a=1):
  return np.pi**2*x*y*np.cos(np.pi*x/2)*np.cos(np.pi*y/2)/2 + np.pi*x*np.sin(np.pi*y/2)*np.cos(np.pi*x/2) + np.pi*y*np.sin(np.pi*x/2)*np.cos(np.pi*y/2)+a*u_ma(x,y)

#print(u_ma(0.5,0.5))
print(f(0.5,0.5))

In [ ]:
# Tamaño de la malla
n=128 #número de particiones
a=1 #parámetro de escala


  # Tamaño de paso
h = 1/n

  # Crear matriz A y vector B
A = np.zeros(((n-1)**2, (n-1)**2))
B = np.zeros((n-1)**2)
  #nodes = np.zeros(((n-1)**2, 2))
nodesx = np.zeros((n-1, n-1))
nodesy = np.zeros((n-1, n-1))
nodesu = np.zeros((n-1, n-1))
errores= np.zeros((n-1, n-1))
# Llenar matriz A y vector B
for i in range(n-1):
    for j in range(n-1):
         index = i*(n-1) + j
         A[index, index] = 4 + a*h**2
         if   i > 0:
           A[index, index - (n-1)] = -1
         if i < n - 2:
           A[index, index + (n-1)] = -1
         if j > 0:
           A[index, index - 1] = -1
         if j < n - 2:
           A[index, index + 1] = -1
           xi = (i + 1) * h
           yi = (j + 1) * h
         nodesx[i , j] = xi
         nodesy[i , j] = yi

         if (xi == 0.5 and yi == 0.5):
           index_mitad = index
         B[index] = h**2 * f(xi, yi, a)
          # Asegúrate de que f(x, y) y a*u_ma(x,y) esté definida correctamente
         #El cambio va a estar en la matriz, recordemos las ecuaciones en dos dimensiones.
         # Resolver el sistema de ecuaciones
U = np.linalg.solve(A, B) #solver lineal para hallar U que es el vector solución
error = np.absolute(U[index_mitad]-u_ma(0.5,0.5))
for i in range(n-1):
    for j in range(n-1):
             index = i*(n-1) + j
             nodesu[i, j] = U[index]



print("Matrix A:")
print(A)
print("Vector B:")
print(B)
print("Solution U:")
print(U)
#print(nodes)

print(u_ma(0.5,0.5)) #valor numérico de la solución manufacturada en x=0.5, y=0.5
print (error)




In [ ]:
#Algoritmo para graficar la solución exacta U en 3D

nodesum=np.zeros((n-1,n-1))

for i in range(n-1):
  for j in range(n-1):
    nodesum[i,j]=u_ma(nodesx[i,j],nodesy[i,j])


fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

surf = ax.plot_surface(nodesx ,nodesy , nodesum , cmap='inferno', linewidth=0, antialiased=False)

#ax.set_zlim(-1.01, 1.01)
ax.zaxis.set_major_locator(LinearLocator(10))

ax.zaxis.set_major_formatter('{x:.02f}')

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()


In [ ]:
tamanoa =[4,16,100,120]
erroresuno =[0.01964698484721189,0.00042770868470370926,5.596053372247667e-06, 3.735829365164589e-06]
erroresdiez = [0.016062311663258597,0.00037140573542232547,4.219881386074986e-06, 2.785259050841482e-06 ]
erroresmenosuno=[0.00845605272006529,0.014274438369693498,0.013750248030834411, 0.013747809099767805]
import matplotlib.pyplot as plt
plt.loglog(tamanoa, erroresuno, label='a=1')
plt.loglog(tamanoa, erroresdiez, label ='a=10')
plt.loglog(tamanoa, erroresmenosuno, label ='a=-1')
plt.xlabel('n')
plt.ylabel('error')
plt.legend()
plt.grid()
plt.show()

In [ ]:
Errores=[]

for i in range(n-1)

In [ ]:

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

surf = ax.plot_surface(nodesx ,nodesy , nodesu , cmap=cm.coolwarm, linewidth=0, antialiased=False)

#plt.plot_surface( nodes[:,1] ,nodes[:,2] , U , cmap=cm.coolwarm, linewidth=0, antialiased=False )


#ax.set_zlim(-1.01, 1.01)
ax.zaxis.set_major_locator(LinearLocator(10))

ax.zaxis.set_major_formatter('{x:.02f}')

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()